# 顔の検出と分析

Computer Vision ソリューションでは、人間の顔を検出、分析、または識別できるようにするために、人工知能 (AI) ソリューションが必要になることがよくあります。たとえば、小売企業 Northwind Traders が「スマート ストア」を開設するシナリオを検討してみます。このスマート ストアでは、AI サービスが店舗を監視して、支援が必要な買い物客を特定し、店員に支援を指示します。これを実現する 1 つの方法は、顔の検出と分析を行うことです。画像に写っている顔があるかどうかを判断し、ある場合はその特徴を分析します。

![顔を分析しているロボット](./images/face_analysis.jpg)

## Face Cognitive Services を使用して顔を検出する

Northwind Traders が構築するスマート ストア システムでは、買い物客を検出して顔の特徴を分析する機能が必要です。Microsoft Azure の Cognitive Services にある **Face** を使用すると、これを行うことができます。

### Cognitive Services リソースを作成する

Azure サブスクリプションに **Cognitive Services** のリソースを作成することから始めます。

> **注**: Cognitive Services リソースが既にある場合は、Azure portal で**クイック スタート**ページを開き、キーとエンドポイントを以下のセルにコピーするだけで作成できます。それ以外の場合は、以下の手順に従って作成してください。

1. ブラウザーの新しいタブで Azure portal (https://portal.azure.com) を開き、Microsoft アカウントでサインインします。
2. 「**&#65291;リソースの作成**」 ボタンをクリックし、*Cognitive Services* を検索して、以下の設定で **Cognitive Services** リソースを作成します。
    - **サブスクリプション**: *使用する Azure サブスクリプション*
    - **リソース グループ**: *一意の名前のリソース グループを選択または作成します*
    - **リージョン**: *利用可能な任意のリージョンを選択します*。
    - **名前**: *一意の名前を入力します*。
    - **価格レベル**: S0
    - **注意事項を読み理解しました**: 選択されています。
3. デプロイが完了するまで待ちます。そのあと Cognitive Services リソースに移動し、「**概要**」 ページでリンクをクリックしてサービスのキーを管理します。クライアント アプリケーションから Cognitive Services リソースに接続するには、エンドポイントとキーが必要です。

### Cognitive Services リソースのキーとエンドポイントを取得する

クライアント アプリケーションで Cognitive Services リソースを使用するには、そのエンドポイントと認証キーが必要です。

1. Azure portalで、Cognitive Services リソースの 「**キーとエンドポイント**」 ページからリソースの 「**キー 1**」 の値をコピーし、以下のコードに貼り付けます (**YOUR_COG_KEY** と置き換える)。

2. リソースの**エンドポイント**をコピーして以下のコードに貼り付けます (**YOUR_COG_ENDPOINT** と置き換える)。

3. セルの左上にある 「セルの実行」 <span>&#9655;</span> ボタンをクリックして、以下のセルのコードを実行します。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

Cognitive Services リソースを作成したので、Face サービスを使用して店舗の中にいる人の顔を検出できるようになりました。

例を確認するには、以下のコード セルを実行してください。

In [ ]:
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from python_code import faces
import os
%matplotlib inline

# Create a face detection client.
face_client = FaceClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Open an image
image_path = os.path.join('data', 'face', 'store_cam2.jpg')
image_stream = open(image_path, "rb")

# Detect faces
detected_faces = face_client.face.detect_with_stream(image=image_stream)

# Display the faces (code in python_code/faces.py)
faces.show_faces(image_path, detected_faces)

検出した各顔には一意の ID が割り当てられるため、検出した個々の顔をアプリケーションで識別できます。

以下のセルを実行して、さらに何人かの買い物客の顔 ID を確認します。

In [ ]:
# Open an image
image_path = os.path.join('data', 'face', 'store_cam3.jpg')
image_stream = open(image_path, "rb")

# Detect faces
detected_faces = face_client.face.detect_with_stream(image=image_stream)

# Display the faces (code in python_code/faces.py)
faces.show_faces(image_path, detected_faces, show_id=True)

## 顔の属性を分析する

Face は、単に顔を検出するだけではありません。顔の特徴や表情を分析して、年齢や感情状態を示唆することもできます。たとえば、以下のコードを実行して、買い物客の顔の属性を分析します。

In [ ]:
# Open an image
image_path = os.path.join('data', 'face', 'store_cam1.jpg')
image_stream = open(image_path, "rb")

# Detect faces and specified facial attributes
attributes = ['age', 'emotion']
detected_faces = face_client.face.detect_with_stream(image=image_stream, return_face_attributes=attributes)

# Display the faces and attributes (code in python_code/faces.py)
faces.show_face_attributes(image_path, detected_faces)

画像から検出した買い物客の感情スコアによると、この客は買い物にかなり満足しているようです。

## 似た顔を見つける 

検出した各顔に付けた顔 ID を使用して、各顔の検出結果を識別します。この ID を使用すると、検出した顔と同じような特徴を持つ顔を、以前に検出した複数の顔の中から見つけることができます。

たとえば、以下のセルを実行して、画像に写っている買い物客と、別の画像に写っている複数の買い物客を比較し、一致する顔を見つけます。

In [ ]:
# Get the ID of the first face in image 1
image_1_path = os.path.join('data', 'face', 'store_cam3.jpg')
image_1_stream = open(image_1_path, "rb")
image_1_faces = face_client.face.detect_with_stream(image=image_1_stream)
face_1 = image_1_faces[0]

# Get the face IDs in a second image
image_2_path = os.path.join('data', 'face', 'store_cam2.jpg')
image_2_stream = open(image_2_path, "rb")
image_2_faces = face_client.face.detect_with_stream(image=image_2_stream)
image_2_face_ids = list(map(lambda face: face.face_id, image_2_faces))

# Find faces in image 2 that are similar to the one in image 1
similar_faces = face_client.face.find_similar(face_id=face_1.face_id, face_ids=image_2_face_ids)

# Show the face in image 1, and similar faces in image 2(code in python_code/face.py)
faces.show_similar_faces(image_1_path, face_1, image_2_path, image_2_faces, similar_faces)

## 顔を認識する

これまで、Face を使用すると、顔とその特徴を検出し、互いに類似している 2 つの顔を識別できることを確認しました。さらに一歩進めるために、Face をトレーニングして特定の人物の顔を認識できる*顔認識*ソリューションを実装します。ソーシャル メディア アプリケーションで友人の写真に自動的にタグを付けたり、生体認証システムの一部として顔認識を使用したりするなど、さまざまなシナリオで役立ちます。

これがどのように機能するかを確認するために、Northwind Traders 社では IT 部門の許可された従業員のみがセキュリティで保護されたシステムにアクセスできるように、顔認識を使用しようとしていると仮定します。

まず、許可された複数の従業員を表す*人物のグループ*を作成します。

In [ ]:
group_id = 'employee_group_id'
try:
    # Delete group if it already exists
    face_client.person_group.delete(group_id)
except Exception as ex:
    print(ex.message)
finally:
    face_client.person_group.create(group_id, 'employees')
    print ('Group created!')

*人物のグループ*を作成したので、このグループに含める従業員ごとに*人物*を追加し、各人物の複数の写真を登録します。これによって、Face が各人物の顔の特徴を学習できるようにします。理想的には、それらの画像は同じ人物をさまざまなポーズや、さまざまな表情で表現している必要があります。

Wendell という従業員を追加し、その従業員の写真を 3 枚登録します。

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

# Add a person (Wendell) to the group
wendell = face_client.person_group_person.create(group_id, 'Wendell')

# Get photo's of Wendell
folder = os.path.join('data', 'face', 'wendell')
wendell_pics = os.listdir(folder)

# Register the photos
i = 0
fig = plt.figure(figsize=(8, 8))
for pic in wendell_pics:
    # Add each photo to person in person group
    img_path = os.path.join(folder, pic)
    img_stream = open(img_path, "rb")
    face_client.person_group_person.add_face_from_stream(group_id, wendell.person_id, img_stream)

    # Display each image
    img = Image.open(img_path)
    i +=1
    a=fig.add_subplot(1,len(wendell_pics), i)
    a.axis('off')
    imgplot = plt.imshow(img)
plt.show()

人物を追加し、写真を登録すると、Face をトレーニングして各人物を認識できるようになります。

In [ ]:
face_client.person_group.train(group_id)
print('Trained!')

これで、モデルをトレーニングできたので、モデルを使用して画像の中から認識した顔を識別できるようになります。

In [ ]:
# Get the face IDs in a second image
image_path = os.path.join('data', 'face', 'employees.jpg')
image_stream = open(image_path, "rb")
image_faces = face_client.face.detect_with_stream(image=image_stream)
image_face_ids = list(map(lambda face: face.face_id, image_faces))

# Get recognized face names
face_names = {}
recognized_faces = face_client.face.identify(image_face_ids, group_id)
for face in recognized_faces:
    person_name = face_client.person_group_person.get(group_id, face.candidates[0].person_id).name
    face_names[face.face_id] = person_name

# show recognized faces
faces.show_recognized_faces(image_path, image_faces, face_names)



## 詳細情報

Face Cognitive Services の詳細については、「[Face ドキュメント](https://docs.microsoft.com/azure/cognitive-services/face/)」を参照してください。
